# Disclaimer

This script relies on scraping data due to the deprecation of the mountain project API in 2020.

Note that scraping data from websites may be against their terms of service, so it's important to check the site's policies before attempting to scrape data. Additionally, scraping large amounts of data or repeatedly scraping a site can put a strain on their servers and may be considered unethical. Always be respectful of the site's resources and policies when scraping data.



In [21]:
import pandas as pd
import plotly.express as px
import datetime as dt
import requests

pd.options.mode.chained_assignment = None  # Disable the warning
# I used venv with this command to get jp working
# ipython kernel install --user --name=your_env_name

"""
(venv) (main) ~/Documents/climb ++ ipython kernel install --user --name=venv
Installed kernelspec venv in /Users/me/Library/Jupyter/kernels/venv
(venv) (main) ~/Documents/climb ++
(venv) (main) ~/Documents/climb ++
(venv) (main) ~/Documents/climb ++
(venv) (main) ~/Documents/climb ++ jupyter-notebook
"""
pass

In [22]:
def insert_newlines(string, splitlength=50):
    if type(string) != str:
        return ""
    return '\n'.join([string[i:i+splitlength] for i in range(0, len(string), splitlength)])


In [23]:
# url = 'https://www.mountainproject.com/user/200169225/seth-drew/tick-export'
url = 'https://www.mountainproject.com/user/200452607/emma-kowal/tick-export'
response = requests.get(url)
df = pd.read_csv(url)


In [24]:
df_codes = pd.read_csv("grade_codes.csv", names=["Rating Code", "Rating Name"]).set_index("Rating Code")
std_rating_names_mapping = df_codes['Rating Name'].to_dict()

In [25]:
df.Rating = df.apply(lambda x: std_rating_names_mapping[x['Rating Code']], axis=1)
df['Date'] = pd.to_datetime(df['Date'])
today = pd.Timestamp.today()
df['Days Ago'] = (today - df['Date']).dt.days
df['Months Ago'] = ((today - df['Date']) / pd.Timedelta(days=30))


In [26]:
sport_code_range = '950 <= `Rating Code` <= 15000'
bouldering_code_range = '20000 <= `Rating Code` <= 22000'

In [27]:
redpoint_df = df.loc[(df['Lead Style'] != 'Fell/Hung') & (df['Style'] != 'Fell/Hung')]
redpoint_df = redpoint_df.drop_duplicates(subset=['Route'])
hungdf = df.loc[(df['Lead Style'] == 'Fell/Hung') | (df['Style'] == 'Fell/Hung')]

# Select only the rows where the "Style" column is "Sport" and the "Rating Code" column falls within the sport grade range
sportdf = redpoint_df.query(sport_code_range)
boulderingdf = redpoint_df.query(bouldering_code_range)


In [28]:
def plot(df):
    df = df.sort_values(['Rating Code', 'Days Ago'])

    fig = px.bar(df, x='Rating', color='Months Ago',
#                  opacity=1-(df['Days Ago']/(df['Days Ago'].max()))**(1/2),
                 barmode='stack', title='Route by Rating, Stacked by Age', 
#                              color_continuous_scale=[(0, 'gray'), (1, 'gray')],
                 hover_data=['Route', "Days Ago", "Notes"])

    fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': df['Rating'].tolist()})

    fig.show()

In [212]:
def animate(df):
    df['Months Ago'] = df['Months Ago'].apply(int)
    fig = px.bar(df, x="Rating",
      animation_frame="Months Ago", barmode='stack')
    fig.show()

SyntaxError: keyword argument repeated: barmode (3086574.py, line 4)

In [210]:
animate(sportdf)

In [30]:
sportdf

,Date,Route,Rating,Notes,URL,Pitches,Location,Avg Stars,Your Stars,Style,Lead Style,Route Type,Your Rating,Length,Rating Code,Days Ago,Months Ago
1,2023-08-01,Predator,5.13b,put it down first go today. everything came to...,https://www.mountainproject.com/route/10588455...,1,New Hampshire > *Rumney > Orange Crush,4.0,4,Lead,Redpoint,Sport,NaN,NaN,8900,20,0.683061
2,2023-07-31,F* *king the Dog,5.11a,"w Serena, great climb!",https://www.mountainproject.com/route/10595169...,1,New Hampshire > *Rumney > Kennel Wall,3.0,3,Lead,Flash,Sport,NaN,50.0,4600,21,0.716395
3,2023-07-30,Orange Crush (retro-bolted),5.9,delightful actually. can't believe I've never ...,https://www.mountainproject.com/route/10677579...,1,New Hampshire > *Rumney > Orange Crush,2.3,-1,Lead,Onsight,Sport,NaN,50.0,2400,22,0.749728
5,2023-07-21,Boardwalk,5.5,wet! but still fun. great views at the top. wi...,https://www.mountainproject.com/route/10590550...,1,"Massachusetts > Central, MA > Crow Hill and Le...",2.4,-1,TR,NaN,"Trad, TR",NaN,80.0,1500,31,1.049728
6,2023-07-21,Outersite,5.7,"first pitch only. delightful! ""like candy"". wi...",https://www.mountainproject.com/route/10660417...,2,"Massachusetts > Central, MA > Crow Hill and Le...",2.2,-1,Lead,Onsight,Trad,NaN,55.0,1800,31,1.049728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2019-04-07,Cardiac Arete (aka route 8),5.10c,NaN,https://www.mountainproject.com/route/11141660...,1,Massachusetts > Western MA > Mormon Hollow,2.5,-1,Lead,Flash,Sport,NaN,40.0,3200,1597,53.249728
506,2018-09-03,Gemini,5.6,second trad lead!,https://www.mountainproject.com/route/10649764...,1,New Hampshire > WM: Kancamagus (Eastern) > Woo...,1.8,-1,Lead,Onsight,Trad,NaN,40.0,1600,1813,60.449728
507,2018-09-03,Zig Zag Crack,5.7,first trad lead!,https://www.mountainproject.com/route/10683718...,1,New Hampshire > WM: Kancamagus (Eastern) > Woo...,1.9,-1,Lead,Onsight,Trad,NaN,40.0,1800,1813,60.449728
508,2018-09-03,Screaming Yellow Zonkers Crack,5.11c,"tried the moves, didn't get very far",https://www.mountainproject.com/route/10672753...,1,New Hampshire > WM: Kancamagus (Eastern) > Woo...,3.9,-1,TR,NaN,Trad,NaN,90.0,5200,1813,60.449728


In [31]:
plot(sportdf)

In [205]:
plot(hungdf)

In [206]:
plot(boulderingdf)

# Reference

In [27]:
## Scraping javascript dynamic search data

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the browser
browser = webdriver.Chrome()
browser.get("https://www.mountainproject.com/search?q=seth-drew&type=users")

# Wait for the data to load
wait = WebDriverWait(browser, 10)
wait.until(EC.presence_of_element_located((By.XPATH, "//img[@class='user-img-avatar']")))

# Get the data
data = browser.page_source

# Clean up
browser.quit()

# Process the data as needed

KeyboardInterrupt: 

In [ ]:
url = 'https://www.mountainproject.com/user/200169225'

# Make a GET request to the URL and retrieve the HTML content
response = requests.get(url)
html_content = response.content